# Rukaten task

I made this notebook more or less as a walkthrough on how I would tackle this problem. 
I have stored non-interesting auxiliary functions in UTILS.py so as not to clutter the notebook. 

First, we import the necessary packages for us to work on the data. I've made a bash script to do that, so we just call it with subprocess. After this, we extract the training data in preparation for the following initial analysis.

In the end, building the model took longer than I thought it would. Prior to actually building it, I was within the three hour mark, but it turned out sklearn's random forest can't handle this data very well, so I switched to pytorch. I think you still get an idea about how I problem solve, even though I prioritized getting a working model over having nice looking visualizations. 

In [1]:
import subprocess
subprocess.call("install_requirements.sh",shell=True)

127

In [2]:
import UTILS

Is this the GPU? GeForce GTX 1650
If not, reset it using reset_gpu()


[nltk_data] Downloading package punkt to /home/k/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
train_data = UTILS.get_data("rdc-catalog-train.tsv",d="\t")

We also extract the test set data, but we will not be looking at it except for seeing its size.

In [4]:
test_data = UTILS.get_data("rdc-catalog-test.tsv",d="\t")

# Initial Exploration

Before preceding, we want look at the training data from various angles. First, of course, the size of the data.

In [5]:
len(train_data)

800000

In [6]:
len(test_data)

200001

In [7]:
# remove the header
header = test_data[0]
test_data = test_data[1:]
print(str(header))

['Title', 'CategoryIdPath']


The training set is certainly large enough for us to partition a validation set out of it. If we do so, we hope to be able to stratify it. But we need to see the distribution of categories to see if that is possible.

As we clean from the project description, data instances consist of two columns, the first being the input description, and the second being the (output) category. The category appears to be hierarchically designated. Let's get a look at how these categories are distributed in the training set... 

In [8]:
#frequencies by category
def get_cat_freqs(set, cat_column = 1):
    cat_freqs = {}
    for cat in [instance[cat_column] for instance in set]:
        if cat in cat_freqs.keys():
            cat_freqs[cat] += 1
        else:
            cat_freqs[cat] = 1
    return cat_freqs

In [9]:
overall_freqs = get_cat_freqs(train_data)
n_cats = len(overall_freqs) # how many unique outputs?
n_cats

3008

In [10]:
import numpy as np
np.mean(list(overall_freqs.values()))
#what is the mean freq of each cat?

265.9574468085106

In [11]:
# how many are "sparse"? Let's say sparse is less than 5 occurrences
sparse_cats = [k for k in overall_freqs if overall_freqs[k] < 5]
len(sparse_cats)

418

In [12]:
# as a proportion? 
len(sparse_cats)/n_cats

0.1389627659574468

In [13]:
# how many are *severely* sparse, i.e. unique? 
len([k for k in overall_freqs if overall_freqs[k] == 1])

19

That's not too bad. It seems we an stratify our train/validation split based on the output variable, so that we can ensure our validation set is representative. We use sklearn's StratifiedKFold for this, treating the 19 unique outputs as all one type "UNK" (for details, consult UTILS.py).

## Target variable structure

### Hierarchical structure

Based on what we have seen in the writeup, it seems that the "category" output is a hierarchical category nesting, with ">" as the delimiter. Before preceding, we confirm to see if this is true. 

In [14]:
# what is the maximum "depth" if we assume ">" delimits levels of a category hierarchy?
max_depth = max([len(line[1].split(">")) for line in train_data+test_data])
max_depth

8

In [15]:
print(""+str(UTILS.get_types_per_level(train_data)))

[  14  108  865 1573  752  232  148    3]


At this point it is still quite plausible that this represents a proper hierarchy; we would assume that many instances simply have a hierarchy that is not deeper than four levels to explain how the numbers taper off after the fourth level. 

If this is a proper hierarchy starting from the first level, there should be no overlap in subtypes (i.e. the higher level types are disjunct). Now we check that...

In [16]:
# first we make a useful function for testing disjunctiveness between a higher and lower category

# @types: list, int, int; @precondition: hi_lvl is less than max_depth -1
# @return: True or False
def cats_disjunct(data, hi_lvl, lvl_difference = 1):
    lo_lvl = hi_lvl + lvl_difference
    subset = [l for l in [line[1].split(">") for line in data] if len(l) > lo_lvl]
    subcats = {}
    hi_lvl_types = UTILS.get_unique_types([l[hi_lvl] for l in subset])
    lo_lvl_types_left = UTILS.get_unique_types([l[lo_lvl] for l in subset])
    for hlt in hi_lvl_types:
        subcats[hlt] = UTILS.get_unique_types([l[lo_lvl] for l in subset if l[hi_lvl] == hlt])
        for llt in subcats[hlt]:
            if llt not in lo_lvl_types_left: #if it has already been hit, that means there is overlap!
                return False
            lo_lvl_types_left.remove(llt)
    return True    

In [17]:
# now we only need to go by steps of 1, since disjunctiveness of subcategories is transitive
# we can go on the whole set here since we aren't really looking at the test data in this way
whole_data = train_data+test_data
for i in range(0, max_depth): 
    print("Does level "+str(i)+" partition level "+str(i+1)+"?\t\t"+str(cats_disjunct(whole_data,i)))

Does level 0 partition level 1?		True
Does level 1 partition level 2?		True
Does level 2 partition level 3?		True
Does level 3 partition level 4?		True
Does level 4 partition level 5?		True
Does level 5 partition level 6?		True
Does level 6 partition level 7?		True
Does level 7 partition level 8?		True


Great, so we have now confirmed that our output labels are hierarchical in nature!

### Getting a sense of what output labels "mean"

So we know our data is hierarchical, but how do the categories branch, and more importantly, can we get a rough sense of how this branching might relate to the textual descriptions? We observe that there are a number of instances with "Toyota" in the description...

In [18]:
toyotas = [ln for ln in train_data if "Toyota" in ln[0]]
len(toyotas)

1337

First we consider a "best case scenario". If "Toyota"'s presence uniquely indicates a single categorical nesting, automatically tagging this should be *trivally easy*.

In [19]:
toyota_cats = get_cat_freqs(toyotas)
len(toyota_cats)

49

Okay, clearly we are not so lucky. Actually, this result is quite disturbing, because this means there is *more* variation among descriptions with "Toyota" in their description than the dataset as a whole (whole: mean 266 instances per category nesting; Toyotas only: only ~27!) 

But maybe the categorical nestings associated with "Toyota" instances do correspond to a pattern that would be very easy to learn. If, say, Toyota's all fall into only 3 categories at the third nesting, this is also a good scenario for automatic classification as there is a good chance that the rest of the description would suffice for disambiguation. So let's see how many unique categories there are at each level for just descriptions including "Toyota". 

In [20]:
print(str(UTILS.get_types_per_level(toyotas)))

[ 7 15 35 36  7]


Well, this is starting to look *really* bad. It is probable that within "Toyotas" there is, instead of any sort of clustering, a data sparsity. But before preceding, let's look at the data itself to see why this might be... 

In [21]:
toyota_head_cats = UTILS.get_unique_types([l[1].split(">")[0] for l in toyotas]) 
for thcat in toyota_head_cats:
    cat_sample_descs = [l[0] for l in toyotas if l[1].split(">")[0]==thcat][:5]
    print("Samples for head category "+str(thcat)+":\n"+"\n".join(cat_sample_descs)+"\n----\n")

Samples for head category 4015:
For Toyota Corolla Black Manual Remote Replacement Driver Side Mirror (TY2709410-ML00)
2-Pack Replacement Engine Air Filter for 2008 Toyota Solara V6 3.3 Car/Automotive
3-Pack Replacement Engine Air Filter for 2000 Toyota Sienna V6 3.0 Car/Automotive
2-Pack Replacement Engine Air Filter for 2008 Toyota Avalon V6 3.5 Car/Automotive
Replacement Cabin Air Filter for 2007 Toyota Corolla L4 1.8 Car/Automotive
----

Samples for head category 2199:
Radiator-1 Row Plastic Tank Aluminum Core CSF 3502 fits 10-11 Toyota Camry
2001-2002-2003 Toyota Rav4 Rav-4 Taillight Taillamp Rear Brake Tail Light Lamp Left Driver Side (01 02 03)
1989 1990 1991 1992 1993 1994 1995 Toyota 4Runner & Toyota Pickup Truck 4WD (without Antenna Hole) Front Fender Quarter Panel (without Molding Holes) Primed/Steel Left Driver Side (89 90 91 92 93 94 95)
For 13-17 Toyota Rav4 4th Gen XA40 Front Bumper Protector Brush Grille Guard (Chrome) 14 15 16
TYC 660101 Toyota Matrix Front Passenger S

This is much more encouraging, as we can see patterns here:

4015: includes air filters and mirrors

3292: all include controls that are usually placed near the right hand of the driver

1395: appear to be all "flashy" products

At the same time, there is also a good deal of variance here, so at this point whether automatically classifying this data is feasible remains an open question. Thus, we analyze what would be the "correct" model to classify this sort of data, and decide on how best to quickly infer whether that model would be successful, by using a fast and relatively light-weight proxy.

# What sort of model would be ideal for this data?

This looks clearly like a hierarchical classification problem to me.
To be honest, I have never done such a problem before. 
With regards to our evaluation metric, I considered two options.
Firstly, I could treat this as a normal classification problem and use an appropriate metric, like F-score (harmonic mean of precision and recall). 
Alternatively, I could look through the literature and find an appropriate metric, under the assumption that getting the parent or grandparent category right counts for at least something. 

Thankfully, Google presented me quickly with what I wanted: a comprehensive literature review on metrics in use in the field of hierarchical text classification. 
I came across the following two papers,
both of which discuss the advantages and disadvantages of different hierarchical classification metrics.
Mercifully, all of the discussed metrics are in some way, like F-score, based on precision and recall, usually combining these two with some sense of "hierarchical distance".

Silla 2011: http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.183.302&rep=rep1&type=pdf

Kosmopoulos 2014:    https://link.springer.com/article/10.1007/s10618-014-0382-x

I was looking for the easiest to implement hierarchical similarity metric.
In addition, I wanted the one that made the least assumptions about the nature of the hierarchy.
After all, I don't even know what the exact categories in this dataset mean as all I can see are their indices in some reference set I don't have access to. 

The first paper, that of Silla 2011, is widely cited, and I found, still being recommended in ML blogs (like this one: https://towardsdatascience.com/https-medium-com-noa-weiss-the-hitchhikers-guide-to-hierarchical-classification-f8428ea1e076). 
For a metric that can be broadly applied, they explicitly recommend "hierarchical F-score" as formulated by Kiritchenko 2005. 

With regards to the exact learning model itself, I think this is less important for our purposes.
The reason we need to know the evaluation metric we want to use is that this is how we tell if our proxy test is indicating that automatic classification will be worth it. 

## Designing our proxy tests

## Assumptions

To be honest, I don't really know all I would like to about this dataset.
For the output variable, all I have is numbers, which I believe are i ndices to categories on some list -- but perhaps that is by design. 
So here are some assumptions I am making about the data and the stakeholders in this scenario.

### Assumption 1 -- high level categories matter more
I assume that the cost of misclassifying a high level category is more than a low category one.
This seems to make sense -- mistaking a Toyota for a Mazda is not ideal, but it is certainly better than mistaking a Toyota for a single Toyota headlight, or even a wine glass or newspaper. 

### Assumption 2 -- it is easier to classify higher level nestings
This is almost always the case, and I imagine it would be for taxonomies of products. What this means for our purposes is that if we cannot perform reasonably well at two-level classification, multi-level classification is not remotely worth it. Likewise, single level "flat" classification failing will be a signal that two-level classification is not worth it. 

### Assumption 3 -- Precision and recall are equally important to stakeholders
More often than not, this is probably the case.

## Our two proposed tests

### Test 1 -- can we learn the top level classifications?
For this, we first see if we can even classify the highest levels to a decent degree of accuracy, as measured by precision (how many labels we get right), recall (how many we got right for each label) and F-score (the harmonic mean of precision and recall). If using a reasonably powerful but simple model we cannot even tell apart the descriptions of Toyota cars from descriptions of antique doorknobs, we can immediately infer that automatic classification will not be worth it. 

In [22]:
from sklearn.metrics import f1_score

# remember to make sure gold_labels and pred_labels are not torch tensors of indexes for labs here...!!
def F1(gold_labels, pred_labels):
    return f1_score(gold_labels, pred_labels, average='macro')

For this task, per assumption 3, our evaluation metric will be a standard F score, and we hope that for the upper level categories, a trained model will perform at 0.8 or higher. For our classification model, we will use a random forest classifier. To quickly tune relevant training parameters, we will use five fold cross-validation using the validation set, and then train on the whole original train set with our best performing setup before evaluating performance on the held-out test set.

If we cannot achieve a decent performance on this task while using a powerful random forest classifier with tuned parameters, then we can safely say that automatic classification is not worth it. If we can learn this well, we will defer to test 2 for a final decision.

### Test 2 -- can we learn the second level categories?

If we perform well enough on task 1, we then want to see if a reasonably powerful model can begin to learn the nesting.

Ideally, one would use a model that learns the hierarchical relationships either explicitly (such as a Bayesian Network classifier) or one specifically designed to learn them implicitly. However, for our purposes, we will use the same random forest classifier, retrained on a task to predict the second nested label, or if there is no nesting, the (single) label. In this way, we are indeed implicitly modeling the nesting, since the first nesting perfectly partitions the second nesting. However, our model is not learning to model the relationship between the hierarchical levels, and it is not impossible that in this scenario it could (incorrectly) label an instance X>A when it is actually Y>B and in test 1 for that instance it would have predicted Y. However, if the data is actually agreeable to automatic classification in a way that hierarchies could be learned in the first place, we should be able to achieve greater precision on test 2 than test 1, so the opposite sort of error (in test 1 predicting X, but in test 2 correctly predicting Y>B) should be far more common. If this is not the case, we don't care, because this actually shows us what we wanted to know -- automatically labeling the data is not a good idea. 

### Some necessary coding to run task 2

As we see below, in terms of how we will stratify the data set for cross-validation, while there are a healthy number of examples for all top level categories in the the training set, the same is not true for second level categories... 

In [23]:
top_level_freqs = get_cat_freqs([['',ln[1].split(">")[0]] for ln in train_data])
print("Frequencies for labels at top level: "+str(top_level_freqs.values()))
second_level_freqs = get_cat_freqs([['',ln[1].split(">")[1] if ">" in ln[1] else ln[1]] for ln in train_data])
print("Frequencies for labels at second level: "+str(second_level_freqs.values()))

Frequencies for labels at top level: dict_values([200945, 28412, 268295, 29557, 96714, 23529, 85554, 18847, 8172, 1030, 5648, 8113, 20086, 5098])
Frequencies for labels at second level: dict_values([33791, 84438, 12743, 57645, 17424, 61787, 41145, 5408, 2295, 2646, 23713, 23322, 77326, 16161, 3637, 4299, 7491, 75129, 9326, 10610, 8172, 369, 6555, 8490, 8291, 12006, 1947, 5653, 12905, 37879, 2476, 2316, 5683, 1347, 12849, 11564, 10498, 1663, 2315, 1794, 222, 2686, 3358, 633, 943, 5376, 7378, 545, 4708, 1757, 553, 116, 2094, 1007, 5530, 2059, 558, 2776, 706, 387, 1419, 7552, 1977, 747, 1653, 64, 1107, 284, 938, 584, 184, 1086, 193, 309, 1791, 504, 1976, 607, 59, 303, 279, 167, 791, 1015, 556, 944, 937, 282, 730, 98, 72, 207, 269, 175, 24, 498, 104, 98, 28, 55, 329, 34, 171, 82, 62, 84, 24, 47, 1])


We need to build a way to recognize the hierarchical relation between the first and second levels anyways for task 2, so this is an opportunity to shoot two birds with one stone.

In [24]:
first_nesting = {}
for (hi, lo)   in [ln[1].split(">")[:2] for ln in train_data if ">" in ln[1]]:
    if hi not in first_nesting:
        first_nesting[hi] = []
    if lo not in first_nesting[hi]:
        first_nesting[hi] += [lo]

def backoff(lo): #recall there is no overlap
    return [k for k,v in first_nesting.items() if lo in v][0]

For this task, we likewise use a neural net that ends in a softmax function, with the same cross-validation process. However, for our evaluation measure will be *Hierarchical F-score* as per Kiritchenko 2005, rather than simple F-score. 
Furthermore, our target is different. Rather than a mere threshold, our goal is to outperform our final scores on test 1 with statistical significance.

In [25]:
lvl1_idx = list(top_level_freqs.keys())
lvl2_idx = list(second_level_freqs.keys()) 
    #these will be useful later.x
task2_lab_idx = lvl1_idx + lvl2_idx
    # the first |len(lvl1-idx)| indicate cases where there was no second level class in the label
len_t2 = len(task2_lab_idx)
    
def get_parent(lvl2_ind):
    return lvl1_idx.index(backoff(lvl2_idx[lvl2_ind-14]))
    
from torch import sum as tsum

# makes onehot vector tensor with 1 at i.
def onehot(i,task2=False):
    out = torch.zeros(len_t2 if task2 else 14)
    out[i] = 1.0
    return out

# oh -- a onehot vector tensor
def onehot_eject(oh):
    return oh.tolist().index(1.)

# gold_labs -- onehot eject used before passing to this class on gold y originally in form of one hot vector tensor, correct value's index is 1.
# pred_labs -- predicted labs, tensor of indices
# indexer is task2_lab_idx -- this works for task1 too since we just ignore index 14 onward. 
def lab_hpr(idx, gold_labs, pred_labs):
    this_true = gold_labs == idx
    this_pred_correct = this_true*(pred_labs == idx)
    if idx < 14 : # no second class in this label, so we are basically doing raw precision.
        return float(tsum(this_pred_correct))/float(tsum(this_true))
    else:
        gold_parents = torch.tensor(
            [li if li < 14 
             else get_parent(li) for li in gold_labs])
        pred_parents = torch.tensor(
            [li if li < 14
             else get_parent(li) for li in pred_labs])
        idx_parent = get_parent(idx)
        
        parent_true = gold_parents == idx_parent
        parent_pred_correct = parent_true*(pred_parents == idx_parent)
        numerator = float(tsum(this_pred_correct)+tsum(parent_pred_correct))
        denominator = float( tsum(this_true)+tsum(parent_true) )
        return numerator/denominator

def hP(g,pr): 
    lab_hPs = torch.tensor([lab_hpr(ci, g, pr)
                            for ci in range(0,len_t2)])
    gold_lab_rates = torch.tensor([torch.sum(g==ci).float()/len_t2 
                                   for ci in range(0,len_t2)])
    return torch.sum(lab_hPs * gold_lab_rates).item()
    
# warning - this will only work if both parameters past are 1d lists/arrays/tensors of int
# gold_labs and pred_labs must be tensors of flat ints (indices of labels), not strings or vectors
# "classes" - list of all possible labels, ints just like above.
def task2_hF(gold_labs, pred_labs):
    h_precision = hP(gold_labs, pred_labs) 
    h_recall = hP(pred_labs,gold_labs)                        
    numerator = 2.0 * h_precision * h_recall 
    denominator = h_precision + h_recall
    return numerator/denominator

# But... what are our features??

Clearly, extracting features from the descriptions is an NLP task. Ideally, we would want to use word embeddings, an attention mechanism, and maybe an LSTM model that takes into account complex sequential relations between tokens. Not a single one of these alone is feasible to do in three hours (at this point of writing I have less than half an hour left -- full disclosure). So instead we extract some basic features. Still, we assert that if a powerful model fails to learn *something* from the following features, automatic classification is not worth it. 

## 1. Unigram features
The presence of each "word" (string token) as a boolean in our description (number of occurrences does not matter). Unigram identity shall not be case sensitive for the first character in a token.

### Frequency threshold and unseen tokens in final evaluation
There is the issue of sparse tokens when using unigram features. For this reason, we introduce a unigram frequency threshold as a model parameter.. Unigrams with frequency beneath the threshold are transformed into "UNK" when calculating unigram features, and do not contribute to classification by the model. Likewise, in prediction stage, any tokens that were unseen in the training set are also immediately transformed into "UNK", and thus do not contribute to prediction. 

## 2. Word shape features
Word shape concerns what sort of characters appear where in a token. This will help us model the presence of serial numbers, etc. We partition possible characters as follows: uppercase alphabetic, lowercase alphabetic, numeric, and every other unique character as its own type (note that spaces, tabs, and line breaks are used as delimiters and thus not included). 

We additionally recruit "word shape summary" features, which compress unbroken sequences of any 3+ instances of character type x into "x*". 

For obvious reasons, our model will not extract word shape features for tokens consisting of only lowercase alphabetic characters. 

Like unigram features, merely having one token with the given word shape will make its corresponding feature value positive, regardless of how many times it occurs. Also like our treatment of unigram features, unseen word shapes do not contribute to prediction at test time.

In [26]:
# some examples of how this works 
print(UTILS.word_shape("abc"))
print(UTILS.word_shape("12abc"))
print(UTILS.word_shape("ANGRY_CAPS!"))
print(UTILS.word_shape("@%#!"))

aaa
nnaaa
AAAAA_AAAA!
@%#!


#### examples
print(UTILS.ws_summary("abc"))
print(UTILS.ws_summary("12abc"))
print(UTILS.ws_summary("ANGRY_CAPS!"))
print(UTILS.ws_summary("@%#!"))

# Experimental design

Now we define our automatic label extraction for the first and second levels:

In [27]:
def lv1_lab (ylab):
    return ylab.split(">")[0]

def lv2_lab (ylab):
    return ylab.split(">")[1] if ">" in ylab else ylab

Now we must set up the experiment. First, we define our model architecture...

In [28]:
import torch.nn as nn
from collections import OrderedDict
import math
class Net(nn.Module):
    def __init__(self, input_width, output_width):
        super(Net,self).__init__()
        widths = [input_width,
                       min(2048, pow(2, 1+int(math.log(input_width,2)))),
                       output_width]
        self.logprob = nn.LogSoftmax(dim=1)

        contents = OrderedDict(); depth = 2
        for i in range(0, depth):
            contents['l'+str(i)] = nn.Linear(widths[i], widths[i+1] , bias=False)
            if i == 0:
                contents['bn'+str(i)] = nn.BatchNorm1d(num_features=widths[i+1])
                contents['a'+str(i)] = nn.LeakyReLU(negative_slope=0.07)
                contents['d'+str(i)] = nn.Dropout(0.2)
        self.network = nn.Sequential(contents)

    def forward(self, x):
        out = self.network(x)
        return out # .squeeze() # this may or may not be necessary to avoid errors.

    def predict(self, input):
        out = self.forward(x)
        # return max probability labels as single ints
        return torch.tensor([torch.argmax(yhi) for yhi in out])
    
    @staticmethod
    def load(model_path):
        if torch.cuda.is_available():
            return torch.load(model_path) if torch.cuda().is_available() else torch.load(model_path, map_location = 'cpu')

    def save(self,path):
            torch.save(self,path)

Then, we decide to use cross entropy loss (hence effectively optimizing while using negative log likelihood) as our objective function while learning.

In [29]:
loss = nn.CrossEntropyLoss()

We then split the training data into a cross-validation set (with 640000 instances -- evenly dividing into minibatches of 1024) and a development set (160000). The development set to halt the final training procedure, while the cv set will be used to determine the optimizer being used and (importantly) the feature threshold. 

In [30]:
import gc

from sklearn.model_selection import StratifiedShuffleSplit
RS = 42 
cv_dev_splitter = StratifiedShuffleSplit(n_splits=1, random_state=RS, test_size = 160000)
cv_inds, dev_inds = [] , []
y_for_split = np.vectorize(lambda x : lv1_lab(train_data[x][1]))(list(range(len(train_data))))
for split in cv_dev_splitter.split(np.zeros_like(y_for_split), y_for_split):
    cv_inds, dev_inds = split
gc.collect()
print("size of cv set "+str(len(cv_inds)))
print("size of dev set "+str(len(dev_inds)))

size of cv set 640000
size of dev set 160000


We will make our cross-validation procedure use a special case of our train procedure, so we define our training procedure first. Before we do that, we define our batching algorithm.

In [31]:
#xparser -- a FeatureExtractor instance
def batch_iter(batch_size, batch_inds, xparser, yparser, test=False, shuffle=True):
    samp_ix = list(range(0,len(test_data))) if test else batch_inds
    data_loc = test_data if test else train_data
    
    if shuffle:
        np.random.shuffle(samp_ix)
    ix_left = len(samp_ix)
    
    while ix_left > 0:
        j, ysamp, xsamp = 0 , [] , []
        while j < batch_size and ix_left > 0:
            xsamp += [data_loc[samp_ix[j]][0]]
            ysamp += [yparser(data_loc[samp_ix[j]][1])]
            j += 1
            ix_left -= 1
        samp_ix = samp_ix[j:]
        xsamp = torch.tensor(np.vectorize(xparser.extract)(xsamp))
        ysamp = torch.tensor([onehot(task2_lab_idx.index(yi)) for yi in ysamp])
        yield ysamp, xsamp

We also need to define our evaluation algorithm before we define our training algorithm.

In [32]:
def evaluate(model, criterion, eval_inds, xparse, yparse, test = False, batch_size = 256):
    model.eval()
    
    running_avg = 0.0
    lines_seen = 0
    
    for Ys, Xs in batch_idx_iter(batch_size, [] if test else eval_inds, 
                                 xparse, yparse, test=test):
        outputs = model.predict(Xs)
        lines_seen += len(outputs)
        running_avg = (prev_lines_seen * running_avg + len(outputs) * 
                       criterion(outputs, Ys ).item()) / lines_seen

    model.train()
    return running_avg    

Now, our training algorithm. 

In [33]:
max_epochs = 5 # we're impatient
patience = 0 #see above. 
def train(feat_xtr, tr_set, validation_set, opt, 
          batch_size = 1024, task2 = False, patience = patience,
          saveloc="last_model.pth", for_crossval = False):
    
    final_epoch = 1 if for_crossval else max_epochs
    curr_epoch = 0
    halt_factor = 1.05 # we're impatient
    
    best_dev_loss = float('inf')
    dev_loss_by_epoch = []
    
    model = Net(feat_xtr.FT_LEN, len(task2_lab_idx) if task2 else 14)
    optimizer = opt(model)
    criterion = loss()
    yparse = [lv1_lab, lv2_lab][int(task2)] 

    model.train()
    halted = False
    
    print("begin training")
    
    while not halted:
        print("Epoch {} ".format(str(curr_epx)))
        curr_batch, print_tot_loss, num_lines = 0 , 0.0 , 0
        print("batching")
    
        for ysmp, xsmp in batch_idx_iter(batch_size, tr_set, feat_xtr, yparse):
            optimizer.zero_grad()
            
            outs = model(xsmp)
            cur_loss = criterion(outs, ysmp)
            
            print_tot_loss += cur_loss * len(y) ; num_lines += len(y)
            
            loss.backward()
            optimizer.step()
            
            del outs; del cur_loss; del x; del y; gc.collect()
            
            curr_batch += 1
            
            if curr_batch % 50 == 0: 
                print("Batch {}, avg loss {}, curr lr {}".format(curr_batch,print_tot_loss / num_lines, optimizer.param_groups[0]['lr']))
                print_tot_loss = num_lines = 0
        
        if cross_val:
            halted = True
            model.save(saveloc)
            model.train()
        else:
            with torch.no_grad():
                dev_loss_by_epoch += [evaluate(model, validation_set)]
    
            print("dev loss "+str(dev_loss_by_epoch[-1]))
        
            halted = halt_factor * dev_loss_by_epoch[-1] > best_dev_loss
            if not halted: 
                best_dev_loss = dev_loss_by_epoch[-1] 
                curr_epoch += 1
            
            if patience > 0:
                patience -= 1
                halted = False
                
            if dev_loss_by_epoch[-1] < best_dev_loss:
                model.save(saveloc)
                model.train()
            
    print("done "+("with cv" if for_crossval else "training"))

We'll cross validate two optimizers:

In [34]:
import torch.optim as optim

def opt_SGD(model):
    return optim.SGD(model.parameters(), lr=0.0001, momentum=0.95, nesterov=nester, weight_decay=0.0000001)
    
def opt_Adamax(model):
    return optim.Adamax(model.parameters())
    
optimizer_options = [ opt_SGD, opt_Adamax ]


And three options for the occurrence threshold for feature inclusion...

In [35]:
threshold_options = [20,50,160]

Now we define our cross-validation class as a series of training runs producing a graph of resulting evaluation metrics, among which we choose the most favorable coordinate. 

In [36]:
    from sklearn.model_selection import StratifiedKFold

In [37]:
n_folds = 5 # both the tr and validation in a fold will be divisible by how powers of 2
rs = 42
# we stratify based on the top level labels -- may be imperfect for lower levels
    # but this is only a proxy test. 
skf = StratifiedKFold(n_splits = n_folds, random_state = rs, shuffle =True)
Y_for_cv = np.vectorize(lv1_lab)([train_data[li][1] for li in cv_inds])
FOLDS = np.array([f for f in enumerate(skf.split(np.zeros_like(Y_for_cv), Y_for_cv))])
print("len train, test in first fold : "+str(len(FOLDS[0][1][0]))+", "+str(len(FOLDS[0][1][1])))

len train, test in first fold : 512000, 128000


In [38]:
def final_eval_f(model, input_inds, xparse, predict_batch_size = 256, task2 = False, test=False):
    ypred = []
    ygold = []
    yparse = [lvl1_lab, lvl2_lab][int(task2)]
    ibi = 0
    if test:
        input_inds = np.array(range(len(test_data)))
    
    while ibi < len(input_inds):
        next_batch_inds = input_inds[ibi*predict_batch_size : (ibi+1)*predict_batch_size]
        next_x = torch.tensor([xparse.extract(train_data[ii][0]) for ii in next_batch_inds])

        ypred = np.concatenate([ypred, model.predict(next_x).tolist()])
        ygold = np.concatenate([ygold, 
            np.vectorize(lambda x : task2_lab_idx.index(yparse(train_data[x][1])))(next_batch_inds)])
        
    return([F1, task2_hF][int(task2)](ypred,ygold))

In [39]:
import os

# returns tuple (x, y) where
    # x -- index of optimal optimization method in optimizer_options
    # y -- index of optimal threshold in threshold_options
def cross_validate(n_folds_to_use = len(FOLDS), # default is all folds but we can speed it things up
                   task2 = False, reuse_paths = True, batch_size = 1024, argbest = np.argmax, 
                  ignore_bigrams = False):
    result_graph = [[0.0 for i in range(0, len(optimizer_options))] for j in range(0, len(threshold_options))]
    
    for fthri in range(0, len(threshold_options)):
        
        feat_extractor = UTILS.FeatureExtractor(
            build_path = "feat_extractor_task"+str(int(task2)+1)+"_th"+str(threshold_options[fthri])+".txt", 
            thresh = threshold_options[fthri])
        if not feat_extractor.feats_determined:
            feat_extractor.build_anew( np.array([l[0] for l in train_data]), ignore_bigrams = ignore_bigrams)
        
        for opti in range(0, len(optimizer_options)):
            scores = []

            print("cross validating for threshold = "+str(threshold_options[fthri])+" with optimizer "+str(opti))
            
            for fi in range(0, n_folds_to_use):
                
                model_path = "cv_t"+("2" if task2 else "1")+"_th"+str(fthri)+"_opt"+str(opti)+".pth"
                if model_path in os.listdir(os.getcwd()) and reuse_paths:
                    print("Reusing crossval trained model with this config to save time.")
                else: 
                    print("Training model anew...")
                    train(feat_extractor, FOLDS[fi][1][0], [], #train algo does not validate when for_crossval = True
                          optimizer_options[opti], batch_size = batch_size, saveloc = model_path, 
                          for_crossval = True)
                
                print("loading...")
                model = torch.load(model_path) if torch.cuda.is_available() \
                    else torch.load(model_path, map_location='cpu')
                
                print("evaluating...")
                scores += [final_eval_f(model, FOLDS[fi][1][1], feat_extractor, task2 = task2)]
                print("F = "+str(scores[-1]))
                
            result_graph[fthri][opti] = np.mean(scores)
        
    return np.unravel_index(argbest(result_graph,axis=None), np.array(result_graph).shape)

The experiment class wraps both task1 and task2. 

In [40]:
class Experiment():
    
    def __init__(self, task2 = False, reuse_path = True):
        print("Beginning task "+str(int(task2)+1))
        self.task2 = task2
        self.reuse_path = reuse_path
        self.model_path = "task"+str(int(task2)+1)+".pth"

    def do_cv(self, cv_folds=5, ignore_bigrams = False):
        if self.reuse_path and self.model_path in os.listdir(os.getcwd()):
            print("Reusing model trained with same configurations from earlier, no need to cross-val")
        else:
            print("Getting cross validation optimizer and feature threshold...")
            self.thi, self.oi = cross_validate(n_folds_to_use = cv_folds, task2 = self.task2, 
                                               ignore_bigrams = ignore_bigrams)

    def extract_features(self):
        self.feat_extractor = UTILS.FeatureExtractor(
            build_path = "feat_extractor_task"+str(int(self.task2)+1)+"_th"+str(threshold_options[self.thi])+".txt", 
            thresh = threshold_options[self.thi])
        if not self.feat_extractor.feats_determined():
            self.feat_extractor.build_anew( np.array([l[0] for l in train_data]))
        
    def final_train(self):
        if self.reuse_path and self.model_path in os.listdir(os.getcwd()):
            print("Reusing model trained with same configurations from earlier...")
        else:
            print("Final training process...")
            train(feat_extractor, cv_data, dev_data, optimizer_options[self.oi], task2 = self.task2, 
                saveloc=self.model_path, for_crossval=False)
        
    def final_eval(self):
        print("loading...")
        model = torch.load(self.model_path) if torch.cuda.is_available() \
            else torch.load(self.model_path, map_location='cpu')        

        print("final evaluation on held-out test set...")
        print("f = "+ str(
              final_eval_f(model, [], self.feat_extractor, predict_batch_size = 64, task2=self.task2, test=True) ))

# Test 1

In [41]:
task1_sim = Experiment()

Beginning task 1


In [ ]:
task1_sim.do_cv(cv_folds=1, ignore_bigrams =True) # just 1 fold and ignore bigrams to save time.

Getting cross validation optimizer and feature threshold...
New feature extractor initialized, but no features yet as no valid path in this directory is specified.
	-- please submit an X set and if necessary a threshhold to build new features
Determining word shape features... (uft = 20)
On 50000th description...
On 100000th description...
On 150000th description...
On 200000th description...
On 250000th description...
On 300000th description...
On 350000th description...
On 400000th description...
On 450000th description...
On 500000th description...
On 550000th description...
On 600000th description...
On 650000th description...
On 700000th description...
On 750000th description...
On 800000th description...
Determining unigram features
On 50000th description...
On 100000th description...
On 150000th description...
On 200000th description...
On 250000th description...
On 300000th description...
On 350000th description...
On 400000th description...
On 450000th description...
On 500000

In [ ]:
task1_sim.extract_features()

In [ ]:
task1_sim.final_train()

In [ ]:
task1_sim.final_eval()

In [ ]:
task2_sim = Experiment(task2=True)

In [ ]:
task2_sim.do_cv(cv_folds=1,ignore_bigrams=True)

In [ ]:
task2_sim.extract_features()